<a href="https://colab.research.google.com/github/sundaybest3/NounSmart/blob/main/nounsmart_irregular_with_datetime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install gradio pandas

In [ ]:
# Import necessary modules
import gradio as gr
import pandas as pd
import random
from datetime import datetime
import pytz  # For time zone handling

# Load the CSV file
csv_url = "https://raw.githubusercontent.com/sundaybest3/NounSmart/refs/heads/main/Irregular_Nouns.csv"
df = pd.read_csv(csv_url)

# Function to get the current date and time in the desired time zone
def get_current_datetime():
    timezone = pytz.timezone("Asia/Seoul")  # Adjust to your desired time zone
    now = datetime.now(timezone)
    return now.strftime("%Y-%m-%d %H:%M:%S")  # Format: YYYY-MM-DD HH:MM:SS

# Other functions remain unchanged
# ...

# Function to check the user's answer
def check_plural(user_plural, user_state):
    if user_state["remaining_nouns"].empty:
        return user_state, "All nouns have been answered correctly. Great job! (Score: {user_state['score']}/{user_state['trials']})", get_current_datetime()

    index = user_state["current_index"]
    if index == -1:
        return user_state, "Please click 'Show the Noun' first. (Score: {user_state['score']}/{user_state['trials']})", get_current_datetime()

    noun_data = user_state["remaining_nouns"].iloc[index]
    singular = noun_data["Singular"]
    correct_plural = noun_data["Plural"]

    # Update trials
    user_state["trials"] += 1

    if user_plural.lower() == correct_plural.lower():
        user_state["score"] += 1
        feedback = f"Correct! '{correct_plural}' is the plural form of '{singular}'."
        # Remove the correctly answered noun
        user_state["remaining_nouns"] = user_state["remaining_nouns"].drop(user_state["remaining_nouns"].index[index])
    else:
        feedback = f"Incorrect. The correct plural form is '{correct_plural}' for '{singular}'. It will appear again."

    if user_state["remaining_nouns"].empty:
        return user_state, f"All nouns have been answered correctly. Great job! (Score: {user_state['score']}/{user_state['trials']})", get_current_datetime()

    return user_state, f"{feedback} (Score: {user_state['score']}/{user_state['trials']})", get_current_datetime()

# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("""
    ### How to Use the App
    1. **Select a CEFR Level to start.**
    2. **Click 'Show the Noun' to display a singular noun.**
    3. **Type its plural form and click 'See the Answer and Feedback' to check.**
    4. **Repeat until all nouns are completed.**
    """)

    # Dropdown for CEFR Level selection
    cefr_dropdown = gr.Dropdown(
        label="Select CEFR Level",
        choices=cefr_levels_with_counts,
        value=cefr_levels_with_counts[0],
        interactive=True
    )
    filter_button = gr.Button("Filter by CEFR Level")
    filter_feedback = gr.Textbox(label="CEFR Level Feedback", interactive=False)

    # Button to display a noun
    show_button = gr.Button("Show the Noun")
    noun_display = gr.Textbox(label="Singular Noun", value="Select a CEFR level to start", interactive=False)

    # Text input for plural form
    plural_input = gr.Textbox(label="Your answer", placeholder="Type your answer here")

    # Submit button and combined feedback + score display
    submit_button = gr.Button("See the Answer and Feedback")
    feedback_display = gr.Textbox(label="Feedback and Score", interactive=False)
    datetime_display = gr.Textbox(label="Current Date and Time", interactive=False)

    # Initialize state
    state = gr.State(initialize_user_state())

    # Button click actions
    filter_button.click(fn=filter_by_cefr, inputs=[cefr_dropdown, state], outputs=[state, filter_feedback])
    show_button.click(fn=show_next_noun, inputs=state, outputs=[state, noun_display, plural_input])
    submit_button.click(fn=check_plural, inputs=[plural_input, state], outputs=[state, feedback_display, datetime_display])

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1ab7e81ebc85211c14.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
